In [ ]:
from matplotlib.pyplot import figure
from problem_formulation import get_model_for_problem_formulation
from ema_workbench import Policy, ema_logging, ScalarOutcome

import pickle
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Define the desired problem formulation here
pf = 3
model, _ = get_model_for_problem_formulation(pf)

In [ ]:
#We do not understand yet why we need a reference_value, these values are from the std document of dike_model_simulation.py

reference_values = {'Bmax': 175, 'Brate': 1.5, 'pfail': 0.5,
                    'discount rate 0': 3.5, 'discount rate 1': 3.5,
                    'discount rate 2': 3.5,
                    'ID flood wave shape': 4}
scen1 = {}
#comparing scenario's with the reference to select scenario's fitted for the problem
for key in model.uncertainties:
    name_split = key.name.split('_')

    if len(name_split) == 1:
        scen1.update({key.name: reference_values[key.name]})

    else:
        scen1.update({key.name: reference_values[name_split[1]]})
        
from ema_workbench import Scenario

ref_scenario = Scenario('reference', **scen1)

### Step 1: Generate optimal policies

In [ ]:
from ema_workbench import MultiprocessingEvaluator

from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)

from ema_workbench.em_framework.evaluators import BaseEvaluator

ema_logging.log_to_stderr(ema_logging.INFO)

#The model is saving the convergence table every 500 nfr, so chose a large number to analyse trends. 

nfe = 100000

#The amount of values for the epsilon list is dependend on the amount of outputs. 
#the larger you choose your epsilon, the more results you will get
#you could chose a lower epsilon in this case. 

len_out = len(model.outcomes.keys()) #This adjusts the size of Epsilon and Hypervolume to the problem formulation
epsilon = [2] * len_out   

#for now there is still a problem with the HyperVolume function. We could decide to not use it because the EpsilonProgress
#is also helpful findig the total amount of policies to establish the convergence of the policie space.

convergence_metrics = [HyperVolume(minimum=[0]*len_out, maximum=[1.01]*len_out),
                       EpsilonProgress()]

#Run ones with Pickle is false to get results, then you could set Pickle on true to avoid unnesecery run time. =) 
use_pickle1 = True
if use_pickle1:
    with open(f"data/MORDM_results1_pf{pf}.pickle","rb") as filehandler:
        results, convergence = pickle.load(filehandler)
else:
    with MultiprocessingEvaluator(model) as evaluator:
        #save it as a Tuple 
        results, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                                  epsilons=epsilon,
                                                  convergence=convergence_metrics,
                                                  reference=ref_scenario,
                                                  n_processes=10)
###!!!!!!
#Change the amount of processes accordingly. 10 processes is recommended for 6core CPU. 14 processes for 8 core CPU        
###!!!!!!
    #Save results in Pickle file
    with open(f"data/MORDM_results1_pf{pf}.pickle","wb") as filehandler:
        pickle.dump((results,convergence),filehandler)

In [ ]:
convergence

In [ ]:
outcomes = results[list(model.outcomes.keys())] #This makes sure that the outcomes are split from the levers/uncertainties 
outcomes.head()

In [ ]:
from ema_workbench.analysis import parcoords

#making a pairplot
import matplotlib.pyplot as plt

limits = parcoords.get_limits(outcomes)
limits.loc[0, outcomes.keys()] = 0

axes = parcoords.ParallelAxes(limits, fontsize=12, rot=90)
axes.plot(outcomes)
#plt.savefig(f"images/ParallelAxes_pf{pf}.svg")
plt.show()

In [ ]:
convergence
#as seen, the hypervolume stays 0, this indicates an error. 

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, figsize=(8,4))
ax1.plot(convergence.nfe, convergence.epsilon_progress)
ax1.set_ylabel('$\epsilon$-progress')
ax2.plot(convergence.nfe, convergence.hypervolume)
ax2.set_ylabel('hypervolume')

ax1.set_xlabel('number of function evaluations')
ax2.set_xlabel('number of function evaluations')
plt.show()

### Step 2: Re-evaluate candidate solutions under uncertainty

In [ ]:
pf = 3
model, _ = get_model_for_problem_formulation(pf)

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

#The model is saving the convergence table every 500 nfr, so chose a large number to analyse trends. 

nfe = 10000

#The amount of values for the epsilon list is dependend on the amount of outputs. 
#the larger you choose your epsilon, the more results you will get
#you could chose a lower epsilon in this case. 

len_out = len(model.outcomes.keys()) #This adjusts the size of Epsilon and Hypervolume to the problem formulation
epsilon = [2] * len_out   

#for now there is still a problem with the HyperVolume function. We could decide to not use it because the EpsilonProgress
#is also helpful findig the total amount of policies to establish the convergence of the policie space.

convergence_metrics = [HyperVolume(minimum=[0]*len_out, maximum=[1.01]*len_out),
                       EpsilonProgress()]

#Run ones with Pickle is false to get results, then you could set Pickle on true to avoid unnesecery run time. =) 
use_pickle1 = True
if use_pickle1:
    with open(f"data/MORDM_results1_pf{pf}.pickle","rb") as filehandler:
        results, convergence = pickle.load(filehandler)
else:
    with MultiprocessingEvaluator(model) as evaluator:
        #save it as a Tuple 
        results, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                                  epsilons=epsilon,
                                                  convergence=convergence_metrics,
                                                  reference=ref_scenario,
                                                  n_processes=10)
###!!!!!!
#Change the amount of processes accordingly. 10 processes is recommended for 6core CPU. 14 processes for 8 core CPU        
###!!!!!!
    #Save results in Pickle file
    with open(f"data/MORDM_results1_pf{pf}.pickle","wb") as filehandler:
        pickle.dump((results,convergence),filehandler)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, figsize=(8,4))
ax1.plot(convergence.nfe, convergence.epsilon_progress)
ax1.set_ylabel('$\epsilon$-progress')
ax2.plot(convergence.nfe, convergence.hypervolume)
ax2.set_ylabel('hypervolume')

ax1.set_xlabel('number of function evaluations')
ax2.set_xlabel('number of function evaluations')
plt.show()

In [ ]:
results["A.3_Expected Number of Deaths"] <= 0.01

In [ ]:
logical = (results["A.3_Expected Number of Deaths"] <= 0.01)
np.sum(logical)

In [ ]:
results[logical].head()

In [ ]:
policies = results[logical]
policies = policies.drop([o.name for o in model.outcomes], axis=1)
policies.head()

In [ ]:
policies_to_evaluate = []

for i, policy in policies.iterrows():
    policies_to_evaluate.append(Policy(str(i), **policy.to_dict()))

In [ ]:
# True, use results in pickle file; False, run MultiprocessingEvaluator
use_pickle2 = True

if use_pickle2:
    with open(f"data/MORDM_results2_pf{pf}.pickle","rb") as filehandler:
        results2 = pickle.load(filehandler)

else:
    # pass the policies list to EMA workbench experiment runs
    n_scenarios = 250
    with MultiprocessingEvaluator(model) as evaluator:
        results2 = evaluator.perform_experiments(n_scenarios, policies_to_evaluate)
    # Save results in Pickle file
    with open(f"data/MORDM_results2_pf{pf}.pickle","wb") as filehandler:
        pickle.dump(results2,filehandler)

In [ ]:
def s_to_n(data, direction):
    mean = np.mean(data)
    std = np.std(data)
    
    if direction==ScalarOutcome.MAXIMIZE:
        return mean/std
    else:
        return mean*std

In [ ]:
experiments2, outcomes2 = results2

overall_scores = {}
for policy in np.unique(experiments2['policy']):
    scores = {}
    
    logical = experiments2['policy']==policy
    
    for outcome in model.outcomes:
        value  = outcomes2[outcome.name][logical]
        sn_ratio = s_to_n(value, outcome.kind)
        scores[outcome.name] = sn_ratio
    overall_scores[policy] = scores
scores = pd.DataFrame.from_dict(overall_scores).T
scores

In [ ]:
data = scores
limits = parcoords.get_limits(data)
limits.loc[0, data.keys()] = 0

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
plt.show()

### Step 3: Give scores using maximum regret

In [ ]:
def calculate_regret(data, best):
    return np.abs(best-data)

In [ ]:
overall_regret = {}
max_regret = {}
for outcome in model.outcomes:
    policy_column = experiments2['policy']
    
    # create a DataFrame with all the relevent information
    # i.e., policy, scenario_id, and scores
    data = pd.DataFrame({outcome.name: outcomes2[outcome.name], 
                         "policy":experiments2['policy'],
                         "scenario":experiments2['scenario']})
    
    # reorient the data by indexing with policy and scenario id
    data = data.pivot(index='scenario', columns='policy')
    
    # flatten the resulting hierarchical index resulting from 
    # pivoting, (might be a nicer solution possible)
    data.columns = data.columns.get_level_values(1)
    
    # we need to control the broadcasting. 
    # max returns a 1d vector across scenario id. By passing
    # np.newaxis we ensure that the shape is the same as the data
    # next we take the absolute value
    #
    # basically we take the difference of the maximum across 
    # the row and the actual values in the row
    #
    outcome_regret = (data.max(axis=1)[:, np.newaxis] - data).abs()
    
    overall_regret[outcome.name] = outcome_regret
    max_regret[outcome.name] = outcome_regret.max()


In [ ]:
max_regret = pd.DataFrame(max_regret)
sns.heatmap(max_regret/max_regret.max(), cmap='viridis', annot=True)
plt.show()

In [ ]:
colors = sns.color_palette()

data = max_regret

# makes it easier to identify the policy associated with each line
# in the parcoords plot
# data['policy'] = data.index.astype("float64")

limits = parcoords.get_limits(data)
limits.loc[0, data.keys()] = 0

paraxes = parcoords.ParallelAxes(limits)
for i, (index, row) in enumerate(data.iterrows()):
    paraxes.plot(row.to_frame().T, label=str(index), color=colors[i])
paraxes.legend()
    
plt.show()

In [ ]:
from collections import defaultdict

policy_regret = defaultdict(dict)
for key, value in overall_regret.items():
    for policy in value:
        policy_regret[policy][key] = value[policy]

In [ ]:
# this generates a 2 by 2 axes grid, with a shared X and Y axis
# accross all plots
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(18,12), 
                         sharey=True, sharex=True)

# to ensure easy iteration over the axes grid, we turn it
# into a list. Because there are four plots, I hard coded
# this. 
axes = [axes[0,0], axes[0,1],
        axes[1,0], axes[1,1]]

# zip allows us to zip together the list of axes and the list of 
# key value pairs return by items. If we iterate over this
# it returns a tuple of length 2. The first item is the ax
# the second items is the key value pair.
for ax, (policy, regret) in zip(axes, policy_regret.items()):
    data = pd.DataFrame(regret)

    # we need to scale the regret to ensure fair visual
    # comparison. We can do that by divding by the maximum regret
    data = data/max_regret.max(axis=0)
    sns.boxplot(data=data, ax=ax)
    
    # removes top and left hand black outline of axes
    sns.despine()
    
    # ensure we know which policy the figure is for
    ax.set_title(str(policy))

    # Rotate x-axis labels slightly to make them redable
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show()

## Multi-scenario MORDM


In [ ]:
y = logical

data = pd.DataFrame({k:v[y] for k,v in outcomes2.items()})
all_data = pd.DataFrame({k:v for k,v in outcomes2.items()})

limits = parcoords.get_limits(all_data)
axes = parcoords.ParallelAxes(limits)
axes.plot(all_data, color='lightgrey')
axes.plot(data, color='blue')
plt.show()


### Here we find diverse scenario's which will test the robustness of our solutions

In [ ]:
from sklearn import preprocessing

experiments_of_interest = experiments2.loc[y]
outcomes_df = pd.DataFrame({k:v[y] for k,v in outcomes2.items()})

# normalize outcomes on unit interval to ensure equal weighting of outcomes
x = outcomes_df.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_outcomes = pd.DataFrame(x_scaled, columns=outcomes_df.columns)

In [ ]:
import itertools

n_scen = experiments2.loc[y].shape[0]
indices = range(int(n_scen))
set_size = 4
n_scen
combinations = itertools.combinations(indices, set_size)
combinations = list(combinations)
len(combinations)

In [ ]:
from scipy.spatial.distance import pdist, squareform

def evaluate_diversity_single(indices, distances, weight=0.5, distance='euclidean'):
    '''
    takes the outcomes and selected scenario set (decision variables), 
    returns a single 'diversity' value for the scenario set.
    outcomes : outcomes dictionary of the scenario ensemble
    decision vars : indices of the scenario set
    weight : weight given to the mean in the diversity metric. If 0, only minimum; if 1, only mean
    '''
    i, j = [e for e in zip(*itertools.combinations(indices, 2))]
    subset_distances = distances[i, j]
    minimum = np.min(subset_distances)
    mean = np.mean(subset_distances)
    diversity = (1-weight)*minimum + weight*mean
    
    return [diversity]


def find_maxdiverse_scenarios(distances, combinations):
    scores = []
    for indices in combinations:
        diversity = evaluate_diversity_single(indices, distances)
        scores.append((diversity, indices))

    return scores

In [ ]:
import random

sampled_combinations = random.sample(sorted(combinations), 100000) #was 100000
#sampled_combinations 

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import os
import functools

distances = squareform(pdist(normalized_outcomes.values))

results = find_maxdiverse_scenarios(distances, sampled_combinations)

In [ ]:
results.sort(key=lambda entry:entry[0], reverse=True)
most_diverse = results[0]
most_diverse

In [ ]:
from ema_workbench import Scenario

selected = experiments2.loc[most_diverse[1], ['A.3_DikeIncrease 0', '0_RfR 0', '1_RfR 0', '2_RfR 0', '3_RfR 0', '4_RfR 0']]
scenarios = [Scenario(f"{index}", **row) for index, row in selected.iterrows()]

In [ ]:
print(len(scenarios))
print(len(model.outcomes))

In [ ]:
#Kwakkels code was cut here

In [ ]:
# True, use results in pickle file; False, run MultiprocessingEvaluator
use_pickle3 = True

if use_pickle3:
    with open(f"data/MORDM_results3_pf{pf}.pickle","rb") as filehandler:
        results3 = pickle.load(filehandler)

else:
    # pass the policies list to EMA workbench experiment runs
    from ema_workbench import MultiprocessingEvaluator, ema_logging
    from ema_workbench.em_framework.evaluators import BaseEvaluator

    from ema_workbench.em_framework.optimization import (HyperVolume,
                                                         EpsilonProgress)

    ema_logging.log_to_stderr(ema_logging.INFO)

    nfe = 1e4 #change to 1e4

    #len_out = len(model.outcomes.keys()) #This adjusts the size of Epsilon and Hypervolume to the problem formulation
    epsilons = [0.2] * 12

    def optimize(scenario, nfe, model, converge_metrics, epsilons):

        with MultiprocessingEvaluator(model) as evaluator:
            results3, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                                      epsilons=epsilons,
                                                      convergence=convergence_metrics,
                                                      reference=ref_scenario)
        return results3, convergence

    results3 = []

    for scenario in scenarios:
        convergence_metrics = [HyperVolume(minimum=[1,1,1,1,1,1,1,1,1,1,1,1], maximum=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]),
                               EpsilonProgress()]
        epsilons = [0.1,]*len(model.outcomes)
        results3.append(optimize(scenario, 1e4, model, convergence_metrics, epsilons)) #is 1e2, was 1e4
        # Save results in Pickle file
    with open(f"data/MORDM_results3_pf{pf}.pickle","wb") as filehandler:
        pickle.dump(results3,filehandler)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
for i, (_, convergence) in enumerate(results3):
    ax1.plot(convergence.nfe, convergence.hypervolume, label=f'scenario {i}')
    ax2.plot(convergence.nfe, convergence.epsilon_progress, label=f'scenario {i}')

ax1.set_ylabel('hypervolume')
ax1.set_xlabel('nfe')
ax2.set_ylabel('$\epsilon$ progress')
ax2.set_xlabel('nfe')
fig.legend()
plt.show()

In [ ]:
from ema_workbench.analysis import parcoords

colors = iter(sns.color_palette())

data = results3[0][0].iloc[:, 31::]
limits = parcoords.get_limits(data)

#limits.loc[0, ['inertia', 'reliability']] = 1
#limits.loc[0, 'max_P'] = 4 # max over results based on quick inspection not shown here
#limits.loc[0, 'utility'] = 1 # max over results based on quick inspection not shown here
#limits.loc[1, :] = 0
paraxes = parcoords.ParallelAxes(limits)


for i, (result, _) in enumerate(results3):
    color = next(colors)
    data = result.iloc[:, ::]
    paraxes.plot(data, label=f'scenario {i}', color=color)

paraxes.legend()
plt.show()


In [ ]:
#results3[2][0].columns

In [ ]:
from ema_workbench import Policy
from ema_workbench import SequentialEvaluator

policies = []
for i, (result, _) in enumerate(results3):
    result = result.iloc[:, 0:31]
    for j, row in result.iterrows():
        policy = Policy(f'scenario {i} option {j}', **row.to_dict())
        policies.append(policy)

In [ ]:
#print(policies)

In [ ]:
# True, use results in pickle file; False, run MultiprocessingEvaluator
use_pickle4 = False

if use_pickle4:
    with open(f"data/MORDM_results4_pf{pf}.pickle","rb") as filehandler:
        reeevaluation_results = pickle.load(filehandler)

else:   
    ema_logging.log_to_stderr(ema_logging.INFO)
        
    with MultiprocessingEvaluator(model) as evaluator:
        reeevaluation_results = evaluator.perform_experiments(2, policies=policies) #was 1000, change to 100
    
        # Save results in Pickle file
    with open(f"data/MORDM_results4_pf{pf}.pickle","wb") as filehandler:
        pickle.dump(reeevaluation_results,filehandler)

In [ ]:
experiments3, outcomes3 = reeevaluation_results

thresholds = {"A.1_Expected Number of Deaths":0.01, "A.2_Expected Number of Deaths":0.01, "A.3_Expected Number of Deaths":0.001, "A.4_Expected Number of Deaths":0.01, "A.5_Expected Number of Deaths":0.01}

overall_scores = {}
for policy in experiments3.policy.unique():
    logical = experiments3.policy == policy
    scores = {}
    for k, v in outcomes3.items():
        try:
            n = np.sum(v[logical]<=thresholds[k])
        except KeyError:
            continue
        scores[k] = n/1000 
    overall_scores[policy] = scores
        
overall_scores = pd.DataFrame(overall_scores).T

In [ ]:
limits = parcoords.get_limits(overall_scores)
print(limits)

In [ ]:
limits = parcoords.get_limits(overall_scores)
paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(overall_scores)
paraxes.legend()
plt.show()

In [ ]:
#a-best section was cut here

In [ ]:
#break

In [ ]:
overall_regret = {}
max_regret = {}
for outcome in model.outcomes:
    policy_column = experiments3['policy']
    
    # create a DataFrame with all the relevent information
    # i.e., policy, scenario_id, and scores
    data = pd.DataFrame({outcome.name: outcomes3[outcome.name], 
                         "policy":experiments3['policy'],
                         "scenario":experiments3['scenario']})
    
    # reorient the data by indexing with policy and scenario id
    data = data.pivot(index='scenario', columns='policy')
    
    # flatten the resulting hierarchical index resulting from 
    # pivoting, (might be a nicer solution possible)
    data.columns = data.columns.get_level_values(1)
    
    # we need to control the broadcasting. 
    # max returns a 1d vector across scenario id. By passing
    # np.newaxis we ensure that the shape is the same as the data
    # next we take the absolute value
    #
    # basically we take the difference of the maximum across 
    # the row and the actual values in the row
    #
    outcome_regret = (data.max(axis=1)[:, np.newaxis] - data).abs()
    
    overall_regret[outcome.name] = outcome_regret
    max_regret[outcome.name] = outcome_regret.max()


In [ ]:
max_regret = pd.DataFrame(max_regret)
sns.heatmap(max_regret/max_regret.max(), cmap='viridis', annot=True)
plt.show()

In [ ]:
colors = sns.color_palette()

data = max_regret

# makes it easier to identify the policy associated with each line
# in the parcoords plot
# data['policy'] = data.index.astype("float64")

limits = parcoords.get_limits(data)
limits.loc[0, data.keys()] = 0

paraxes = parcoords.ParallelAxes(limits)
for i, (index, row) in enumerate(data.iterrows()):
    paraxes.plot(row.to_frame().T, label=str(index))
paraxes.legend()
    
plt.show()

In [ ]:
# this generates a 2 by 2 axes grid, with a shared X and Y axis
# accross all plots
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(18,12), 
                         sharey=True, sharex=True)

# to ensure easy iteration over the axes grid, we turn it
# into a list. Because there are four plots, I hard coded
# this. 
axes = [axes[0,0], axes[0,1],
        axes[1,0], axes[1,1]]

# zip allows us to zip together the list of axes and the list of 
# key value pairs return by items. If we iterate over this
# it returns a tuple of length 2. The first item is the ax
# the second items is the key value pair.
for ax, (policy, regret) in zip(axes, policy_regret.items()):
    data = pd.DataFrame(regret)

    # we need to scale the regret to ensure fair visual
    # comparison. We can do that by divding by the maximum regret
    data = data/max_regret.max(axis=0)
    sns.boxplot(data=data, ax=ax)
    
    # removes top and left hand black outline of axes
    sns.despine()
    
    # ensure we know which policy the figure is for
    ax.set_title(str(policy))

    # Rotate x-axis labels slightly to make them redable
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show()